In [5]:
# imports
from pathlib import Path
import urllib.request
import pandas as pd
from google.cloud import bigquery
from prefect import task, flow
from prefect_gcp.cloud_storage import cloud_storage_upload_blob_from_file
from prefect_gcp import GcpCredentials

print('Setup Complete')

Setup Complete


In [4]:
# Parameters
year=2019
month=2
dataset_file = f"fhv_tripdata_{year}-{month:02}"
dataset_url = f"https://github.com/DataTalksClub/nyc-tlc-data/releases/download/fhv/{dataset_file}.csv.gz"

# Deployment 1
# Fetch the data from url
@task(log_prints=True, name='fetch-file-dataset_url')
def fetch(dataset_url):
    filename, _ = urllib.request.urlretrieve(dataset_url)
    return filename


# Upload dataset from web to gcs
@flow(log_prints=True, name="web-to-gcs")
def web_to_gcs(path: str, year: int, month: int):
    gcp_credentials = GcpCredentials.load("ny-taxi-gcp-creds")  # inferred from env, or set service_account_file
    blob = cloud_storage_upload_blob_from_file(file=path, 
                                                bucket="ny_taxi_bucket_de_2023",
                                                blob=f"2019/fhv_tripdata_{year}-{month:02}.csv.gz",
                                                gcp_credentials=gcp_credentials)
    return blob



In [6]:
@flow()
def etl_web_gcs_bq():
    path = fetch(dataset_url)
    web_to_gcs(path, year=2019, month=2)
    # fetch data from github
    # save data to gcs
    # transfer data to bq

if __name__=="__main__":
 etl_web_gcs_bq()   


09:52:23.632 | INFO    | prefect.engine - Created flow run 'victorious-oryx' for flow 'etl-web-gcs-bq'

09:52:25.868 | INFO    | Flow run 'victorious-oryx' - Created task run 'fetch-file-dataset_url-b4598a4a-0' for task 'fetch-file-dataset_url'

09:52:25.875 | INFO    | Flow run 'victorious-oryx' - Executing 'fetch-file-dataset_url-b4598a4a-0' immediately...

09:52:45.426 | INFO    | Task run 'fetch-file-dataset_url-b4598a4a-0' - Finished in state Completed()

09:52:46.358 | INFO    | Flow run 'victorious-oryx' - Created subflow run 'naughty-parrot' for flow 'web-to-gcs'

...

09:52:47.581 | INFO    | Flow run 'naughty-parrot' - Executing 'cloud_storage_upload_blob_from_file-9a1bf371-0' immediately...

09:52:48.569 | INFO    | Task run 'cloud_storage_upload_blob_from_file-9a1bf371-0' - Uploading blob named 2019/fhv_tripdata_2019-02.csv.gz to the ny_taxi_bucket_de_2023 bucket

09:52:58.502 | INFO    | Task run 'cloud_storage_upload_blob_from_file-9a1bf371-0' - Finished in state Completed()

09:52:59.267 | INFO    | Flow run 'naughty-parrot' - Finished in state Completed()

09:52:59.601 | INFO    | Flow run 'victorious-oryx' - Finished in state Completed('All states completed.')

In [8]:
# Deployment 2
# load GCP Credentials
@task(log_prints=True, name="get-gcp-creds")
def get_bigquery_creds():
    gcp_creds_block = GcpCredentials.load("prefect-gcs-2023-creds")
    gcp_creds = gcp_creds_block.get_credentials_from_service_account()
    return gcp_creds


# Upload data from GCS to BigQuery
@flow(log_prints=True, name="etl-gcs-to-bq")
def etl_gcs_to_bq(year: int, month: int):

    gcp_creds = get_bigquery_creds()
    client = bigquery.Client(credentials=gcp_creds)
    table_id = "dtc-de-2023.ny_taxi.ny_taxi_tripdata_2019"

    job_config = bigquery.LoadJobConfig(
        source_format=bigquery.SourceFormat.CSV,
        schema=[
            bigquery.SchemaField("dispatching_base_num", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("pickup_datetime", "DATETIME", mode="NULLABLE"),
            bigquery.SchemaField("dropOff_datetime", "DATETIME", mode="NULLABLE"),
            bigquery.SchemaField("PUlocationID", "FLOAT", mode="NULLABLE"),
            bigquery.SchemaField("DOlocationID", "FLOAT", mode="NULLABLE"),
            bigquery.SchemaField(
                "SR_Flag",
                "FLOAT",
                mode="NULLABLE",
            ),
            bigquery.SchemaField("Affiliated_base_number", "STRING", mode="NULLABLE"),
        ],
    )
    uri = f"gs://ny_taxi_bucket_de_2023/2019/fhv_tripdata_{year}-{month:02}.csv.gz"

    load_job = client.load_table_from_uri(
        uri, table_id, job_config=job_config
    )  # Make an API request.

    load_job.result()  # Waits for the job to complete.

    destination_table = client.get_table(table_id)
    print(f"Loaded {destination_table.num_rows} rows.")


# Parent flow ETL
@flow(log_prints=True, name="etl-parent-to-bq")
def etl_parent_bq_flow(
    year: int = 2019, months: list[int] = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
):
    for month in months:
        etl_gcs_to_bq(year, month)


# run main
if __name__ == "__main__":
    year = 2019
    months = [2]

    etl_parent_bq_flow(year, months)


/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/prefect/tasks.py:261: UserWarning: A task named 'get-gcp-creds' and defined at '/var/folders/h2/6110hgtx6b7400ybw7yjhn400000gn/T/ipykernel_12132/2370067010.py:3' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/prefect/flows.py:206: UserWarning: A flow named 'etl-gcs-to-bq' and defined at '/var/folders/h2/6110hgtx6b7400ybw7yjhn400000gn/T/ipykernel_12132/2370067010.py:11' conflicts with another flow. Consider specifying a unique `name` parameter in the flow definition:

 `@flow(name='my_unique_name', ...)`
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/prefect/flows.py:206: UserWarning: A flow named 'etl-parent-to-bq' and defined at '/var/folders/h2/6110hgtx6b7400ybw7yjhn400000gn/T/ipykernel_12132/2370067010.py:47' con

10:03:28.172 | INFO    | prefect.engine - Created flow run 'brave-bat' for flow 'etl-parent-to-bq'

/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/prefect/tasks.py:261: UserWarning: A task named 'etl-gcs-to-bq' and defined at '/var/folders/h2/6110hgtx6b7400ybw7yjhn400000gn/T/ipykernel_12132/2370067010.py:11' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(


10:03:31.249 | INFO    | Flow run 'brave-bat' - Created subflow run 'original-vole' for flow 'etl-gcs-to-bq'

10:03:32.484 | INFO    | Flow run 'original-vole' - Created task run 'get-gcp-creds-b52b4cc6-0' for task 'get-gcp-creds'

...

10:03:36.383 | ERROR   | Flow run 'original-vole' - Encountered exception during execution:

10:03:36.826 | ERROR   | Flow run 'original-vole' - Finished in state Failed("Flow run encountered an exception. google.api_core.exceptions.BadRequest: 400 Error while reading data, error message: Could not parse 'pickup_datetime' as TIMESTAMP for field pickup_datetime (position 1) starting at location 0  with message 'Could not parse 'pickup_datetime' as a timestamp. Required format is YYYY-MM-DD HH:MM[:SS[.SSSSSS]] or YYYY/MM/DD HH:MM[:SS[.SSSSSS]]' File: gs://ny_taxi_bucket_de_2023/2019/fhv_tripdata_2019-02.csv.gz\n")

10:03:36.829 | ERROR   | Flow run 'brave-bat' - Encountered exception during execution:
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/prefect/engine.py", line 643, in orchestrate_flow_run
    result = await run_sync(flow_call)
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/prefect/utilities/asyncutils.py", line 156, in run_sync_in_interruptible_worker_thread
    tg.start_soon(
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/anyio/_backends/_asyncio.py", line 662, in __aexit__
    raise exceptions[0]
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/anyio/to_thread.py", line 31, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/anyio/_backends/_asyncio.py", line 937, in run_sync_in_worker_thread
    return await future
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/anyio/_backends/_asyncio.py", line 867, in run
    result = context.run(func, *args)
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/prefect/utilities/asyncutils.py", line 135, in capture_worker_thread_and_result
    result = __fn(*args, **kwargs)
  File "/var/folders/h2/6110hgtx6b7400ybw7yjhn400000gn/T/ipykernel_12132/2370067010.py", line 52, in etl_parent_bq_flow
    etl_gcs_to_bq(year, month)
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/prefect/flows.py", line 448, in __call__
    return enter_flow_run_engine_from_flow_call(
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/prefect/engine.py", line 175, in enter_flow_run_engine_from_flow_call
    return run_async_from_worker_thread(begin_run)
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/prefect/utilities/asyncutils.py", line 177, in run_async_from_worker_thread
    return anyio.from_thread.run(call)
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/anyio/from_thread.py", line 49, in run
    return asynclib.run_async_from_thread(func, *args)
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/anyio/_backends/_asyncio.py", line 970, in run_async_from_thread
    return f.result()
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/concurrent/futures/_base.py", line 446, in result
    return self.__get_result()
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/concurrent/futures/_base.py", line 391, in __get_result
    raise self._exception
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/prefect/client/utilities.py", line 47, in with_injected_client
    return await fn(*args, **kwargs)
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/prefect/engine.py", line 553, in create_and_begin_subflow_run
    return await terminal_state.result(fetch=True)
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/prefect/states.py", line 89, in _get_state_result
    raise await get_state_exception(state)
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/prefect/engine.py", line 643, in orchestrate_flow_run
    result = await run_sync(flow_call)
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/prefect/utilities/asyncutils.py", line 156, in run_sync_in_interruptible_worker_thread
    tg.start_soon(
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/anyio/_backends/_asyncio.py", line 662, in __aexit__
    raise exceptions[0]
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/anyio/to_thread.py", line 31, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/anyio/_backends/_asyncio.py", line 937, in run_sync_in_worker_thread
    return await future
  File "/o

10:03:37.140 | ERROR   | Flow run 'brave-bat' - Finished in state Failed("Flow run encountered an exception. google.api_core.exceptions.BadRequest: 400 Error while reading data, error message: Could not parse 'pickup_datetime' as TIMESTAMP for field pickup_datetime (position 1) starting at location 0  with message 'Could not parse 'pickup_datetime' as a timestamp. Required format is YYYY-MM-DD HH:MM[:SS[.SSSSSS]] or YYYY/MM/DD HH:MM[:SS[.SSSSSS]]' File: gs://ny_taxi_bucket_de_2023/2019/fhv_tripdata_2019-02.csv.gz\n")

BadRequest: 400 Error while reading data, error message: Could not parse 'pickup_datetime' as TIMESTAMP for field pickup_datetime (position 1) starting at location 0  with message 'Could not parse 'pickup_datetime' as a timestamp. Required format is YYYY-MM-DD HH:MM[:SS[.SSSSSS]] or YYYY/MM/DD HH:MM[:SS[.SSSSSS]]' File: gs://ny_taxi_bucket_de_2023/2019/fhv_tripdata_2019-02.csv.gz